## Запрос №3
Напишите запрос, выбирающий информацию по всем служащим, 
нанятым раньше своих менеджеров. При этом выведите фамилии и даты 
найма самих служащих, а также фамилии и даты найма их менеджеров. 
Пример возможного результата выполнения запроса представлен в 
таблице ниже. Поля должны называться так же, как указано в примере.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType

spark = SparkSession.builder.master('local').getOrCreate()

empDf = spark.read.parquet('data/employees.parquet')

cmp = udf(lambda a, b: tuple(map(int, a.split('.'))) < tuple(map(int, b.split('.'))), BooleanType())

df = empDf.withColumnRenamed("last_name", "Фамилия_Р").withColumnRenamed("hire_date", "dt_w")
df2 = empDf.select("employee_id", "last_name", "hire_date")
df = df.join(df2, df.manager_id == df2.employee_id, "inner") \
    .where(cmp(col("dt_w"), col("hire_date"))).withColumnRenamed("last_name", "Фамилия_М") \
    .withColumnRenamed("hire_date", "Дата_М").withColumnRenamed("dt_w", "Дата_Р")
df.select("Фамилия_Р", "Дата_Р", "Фамилия_М", "Дата_М").show(truncate=False)


+---------+----------+---------+----------+
|Фамилия_Р|Дата_Р    |Фамилия_М|Дата_М    |
+---------+----------+---------+----------+
|De Haan  |13.01.2001|King     |17.06.2003|
|Hunold   |03.01.2006|De Haan  |13.01.2001|
|Greenberg|17.08.2002|Kochhar  |21.09.2005|
|Faviet   |16.08.2002|Greenberg|17.08.2002|
|Urman    |07.03.2006|Greenberg|17.08.2002|
|Popp     |07.12.2007|Greenberg|17.08.2002|
|Raphaely |07.12.2002|King     |17.06.2003|
|Fripp    |10.04.2005|King     |17.06.2003|
|Kaufling |01.05.2003|King     |17.06.2003|
|Vollman  |10.10.2005|King     |17.06.2003|
|Mourgos  |16.11.2007|King     |17.06.2003|
|Nayer    |16.07.2005|Weiss    |18.07.2004|
|Landry   |14.01.2007|Weiss    |18.07.2004|
|Markle   |08.03.2008|Weiss    |18.07.2004|
|Patel    |06.04.2006|Vollman  |10.10.2005|
|Matos    |15.03.2006|Mourgos  |16.11.2007|
|Vargas   |09.07.2006|Mourgos  |16.11.2007|
|Russell  |01.10.2004|King     |17.06.2003|
|Partners |05.01.2005|King     |17.06.2003|
|Errazuriz|10.03.2005|King     |